In [8]:
from gensim.models import Word2Vec
import MeCab
def tokenize(text):
    """ テキストを形態素解析して、トークンのリストを返す """
    mecab = MeCab.Tagger("-Owakati")
    return mecab.parse(text).strip().split()
with open('./Data/dokujo-tsushin.txt', 'r', encoding='utf-8') as file:
    corpus = [line.strip() for line in file if line.strip()]  # 空白行をスキップ


In [10]:
tokenized_documents = [tokenize(doc) for doc in corpus]

In [12]:
model_jp = Word2Vec(sentences=tokenized_documents, vector_size=100, window=5, min_count=1, workers=4)

In [14]:
model_jp.wv['日本']

array([-0.4123209 ,  0.55828583,  0.15993343, -0.33039883, -0.10033111,
       -1.093515  ,  0.32533425,  0.8549594 , -0.7223899 , -0.528165  ,
       -0.64176565, -0.5838085 ,  0.55550796,  0.13387133, -0.32790843,
       -0.9167322 ,  0.23246412, -1.0256983 , -1.1781516 , -0.3665961 ,
       -0.08638341,  0.3929385 , -0.05359071, -0.41832042,  0.7190247 ,
       -0.07000482,  0.41131148, -0.12937541, -0.7119168 , -0.07596931,
       -0.37757674, -0.09149896,  0.68786234, -0.58384156, -0.68312204,
        0.33096346,  0.22713119, -0.7432019 ,  0.06821846, -0.51077867,
       -0.21847431, -0.4603345 ,  0.12346501, -0.5041283 ,  0.6162513 ,
        0.23929003, -0.55735505, -0.01441303,  0.5277629 , -0.02835805,
       -0.03092367, -0.01384843, -0.4187761 ,  0.40490156, -0.80793923,
        0.45574433,  0.5846862 ,  0.344196  ,  0.44368654, -0.05931788,
        0.6585717 ,  0.37178415, -0.19977711,  0.18319508, -0.05026088,
        1.0419227 , -0.35501447, -0.1297323 , -0.6497035 ,  0.46

In [16]:
model_jp.wv.most_similar("男")

[('女', 0.9335235953330994),
 ('友達', 0.8225789666175842),
 ('モテ', 0.7815700173377991),
 ('男性', 0.768693745136261),
 ('恋人', 0.7675696015357971),
 ('浮気', 0.748518705368042),
 ('彼女', 0.7295669913291931),
 ('理想', 0.724653422832489),
 ('すさん', 0.7177618741989136),
 ('あなた', 0.7163294553756714)]

In [18]:
import gensim.downloader

print(list(gensim.downloader.info()['models'].keys()))
model = gensim.downloader.load('word2vec-google-news-300')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [20]:
# ベクトル演算を行う
result_vector = model['king'] - model['man'] + model['woman']

# 類似度の高い10語を取得
similar_words = model.most_similar(positive=[result_vector], topn=10)

# 結果を出力
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

king: 0.8449
queen: 0.7301
monarch: 0.6455
princess: 0.6156
crown_prince: 0.5819
prince: 0.5777
kings: 0.5614
sultan: 0.5377
Queen_Consort: 0.5344
queens: 0.5290


In [19]:

# ベクトル演算を行う
result_vector = model.wv['king'] - model.wv['man'] + model.wv['woman']

# 類似度の高い10語を取得
similar_words = model.wv.most_similar(positive=[result_vector], topn=10)

# 結果を出力
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")

AttributeError: 'KeyedVectors' object has no attribute 'wv'

In [ ]:
male_list=["man","men","his","his","he","male","masculine"]
female_list=["woman","women","her","hers","she","female","feminine"]

In [ ]:
import numpy as np
male_vec=[]
for i,j in zip(male_list,female_list):
    male_vec.append(model[i]-model[j])
male_vec=np.array(male_vec)
male_vec=np.mean(male_vec,axis=0)

In [ ]:
for sport in sports:
    print(sport,get_angle(model[sport],male_vec,degree=True))

In [25]:
import numpy as np
from gensim.downloader import load
from numpy.linalg import norm

# モデルの読み込み
model = load("word2vec-google-news-300")

# 性別を代表する単語リスト
female_words = ["she", "female", "woman", "girl"]
male_words = ["he", "male", "man", "boy"]

# 職業単語リスト
occupation_words = ["engineer", "nurse", "housekeeper"]

# 性別バイアスを計算する関数
def calculate_gender_bias(occupation, female_words, male_words, model):
    # 距離の計算 (女性)
    female_distances = [norm(model[occupation] - model[female]) for female in female_words]
    female_mean = np.mean(female_distances)

    # 距離の計算 (男性)
    male_distances = [norm(model[occupation] - model[male]) for male in male_words]
    male_mean = np.mean(male_distances)

    # 性別バイアス
    gender_bias = female_mean - male_mean
    return gender_bias

# 各職業の性別バイアスを計算
bias_results = {}
for occupation in occupation_words:
    bias = calculate_gender_bias(occupation, female_words, male_words, model)
    bias_results[occupation] = bias



In [24]:
# 結果を表示
for occupation, bias in bias_results.items():
    relation = "closer to women" if bias < 0 else "closer to men"
    print(f"Gender Bias for {occupation}: {bias:.4f} ({relation})")

Gender Bias for engineer: 0.2160 (closer to men)
Gender Bias for nurse: -0.2625 (closer to women)
Gender Bias for housekeeper: -0.1960 (closer to women)
